## Q1. Running Elastic 

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

## A1.
The `version.build_hash` is "42f05b9372a9a4a470db3b52817899b99a76ee73"


## Getting the data

Now let's get the FAQ data. You can run this snippet:

In [10]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
documents_raw = requests.get(docs_url).json()

documents = []
for course in documents_raw:
    course_name = course['course']
    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [4]:
# documents
# documents.sort(key=lambda x: x['course'])

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?


In [5]:
from  elasticsearch import Elasticsearch

esc = Elasticsearch("http://localhost:9200")

In [11]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

# Delete existing index
es.indices.delete(index='zoomcamp_faq', ignore_unavailable=True)

# Create index with mapping
es.indices.create(
    index='zoomcamp_faq',
    body={
        "mappings": {
            "properties": {
                "course": {"type": "keyword"},
                "question": {"type": "text"},
                "text": {"type": "text"}
            }
        }
    }
)

# Index the data
for i, doc in enumerate(documents):
    es.index(index='zoomcamp_faq', id=i, document=doc)


/tmp/ipykernel_55307/151206867.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(


## A2.

The function I used for adding the data into elasticsearch is "index".

## Q3. Searching

Now let's search in our index. 

We will execute a query "How do execute a command on a Kubernetes pod?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

In [12]:
query = {
    "query": {
        "multi_match": {
            "query": "How do execute a command on a Kubernetes pod?",
            "fields": ["question^4", "text"],
            "type": "best_fields"
        }
    }
}

res = es.search(index="zoomcamp_faq", body=query)
print(res["hits"]["hits"][0]["_score"])


44.50556


/tmp/ipykernel_55307/3232417081.py:11: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="zoomcamp_faq", body=query)


## A3.

Top ranking score is "44.5"

## Q4. Filtering

Now ask a different question: "How do copy a file to a Docker container?".

This time we are only interested in questions from `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

In [13]:
query = {
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": "How do copy a file to a Docker container?",
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {"course": "machine-learning-zoomcamp"}
            }
        }
    }
}

res = es.search(index="zoomcamp_faq", body=query, size=3)
for hit in res['hits']['hits']:
    print(hit['_source']['question'])


How do I debug a docker container?
How do I copy files from my local machine to docker container?
How do I copy files from a different folder into docker container’s working directory?


/tmp/ipykernel_55307/3152046346.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="zoomcamp_faq", body=query, size=3)


## A4.

3rd question is "How do I copy files from a different folder into docker container’s working directory?"

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do copy a file to a Docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

In [14]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join([
    context_template.format(**hit['_source']) for hit in res['hits']['hits']
])

question = "How do copy a file to a Docker container?"

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=question, context=context)
print(len(prompt))


1446


## A5. 

The Leght of result prompt is "1446".

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

In [16]:
# !pip install tiktoken


In [17]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(prompt)
print(len(tokens))


320


## A6.

Our prompt has "320" tokens.

## A7 (Bonus)


Let’s say:

1000 requests

150 input tokens

250 output tokens


Prices:

Input: $0.005 / 1K tokens

Output: $0.015 / 1K tokens

In [18]:
input_cost = (1000 * 150 / 1000) * 0.005  # = $0.75
output_cost = (1000 * 250 / 1000) * 0.015  # = $3.75
total = input_cost + output_cost
print(total)


4.5
